In [1]:
import pandas as pd
import numpy as np

In [2]:
movie = pd.read_csv(r'C:\Users\HPPC\Downloads\imdb-5000-movie-dataset\movie_metadata.csv')

In [4]:
movie.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [11]:
movie.isnull().sum()

color                         19
director_name                  0
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                   0
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   0
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                   0
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

### Content Based - Similarity between Items

In [10]:
#Actors and director names- missing values with unknown
movie['actor_1_name'] = movie['actor_1_name'].fillna('Unknown')
movie['actor_2_name'] = movie['actor_2_name'].fillna('Unknown')
movie['actor_3_name'] = movie['actor_3_name'].fillna('Unknown')
movie['director_name'] = movie['director_name'].fillna('Unknown')

In [18]:
#replace | with space
movie['genres'] = movie['genres'].str.replace('|'," ")
movie['genres'].value_counts()

Drama                                                         236
Comedy                                                        209
Comedy Drama                                                  191
Comedy Drama Romance                                          187
Comedy Romance                                                158
Drama Romance                                                 152
Crime Drama Thriller                                          101
Horror                                                         71
Action Crime Drama Thriller                                    68
Action Crime Thriller                                          65
Drama Thriller                                                 64
Crime Drama                                                    63
Horror Thriller                                                56
Crime Drama Mystery Thriller                                   55
Action Adventure Sci-Fi                                        51
Documentar

In [20]:
#converting movie title into lower string for simplicity
movie['movie_title'] = movie['movie_title'].str.lower()

In [28]:
#Removing extension
movie['movie_title'] = movie['movie_title'].str[:-1] 

In [35]:
movie['movie_title'][9]

'harry potter and the half-blood prince'

In [36]:
#Combining features into One column - Helps to identify similar movie
movie['comb'] = movie['actor_1_name']+movie['actor_2_name']+movie['actor_3_name']+movie['director_name']+movie['genres']

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
#creating count matrix - Embedding using CountVectorize 
cv = CountVectorizer()
count_matrix = cv.fit_transform(movie['comb'])

#finding cosine similarity in count matrix
sim = cosine_similarity(count_matrix)

In [44]:
sim

array([[1.        , 0.23904572, 0.11952286, ..., 0.        , 0.        ,
        0.        ],
       [0.23904572, 1.        , 0.14285714, ..., 0.        , 0.        ,
        0.        ],
       [0.11952286, 0.14285714, 1.        , ..., 0.14285714, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.14285714, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [45]:
sim.shape

(5043, 5043)

In [101]:
#movie of user choice
m = 'Star trek'
m = m.lower()
m

'star trek'

In [102]:
m in movie['movie_title'].unique()

True

In [103]:
#find index where movie title is present to find the features it has
movie_index = movie.loc[movie['movie_title']==m].index[0]

In [104]:
movie_index

160

In [105]:
#create list having movie index and similarity score from similarity matrix
movie_list = list(enumerate(sim[movie_index]))

In [106]:
len(movie_list)

5043

In [107]:
#sort movie in descending order based on score
movie_list = sorted(movie_list,key = lambda x : x[1],reverse=True)[1:11]

In [108]:
movie_list

[(48, 0.4999999999999999),
 (171, 0.4999999999999999),
 (222, 0.4714045207910316),
 (8, 0.4472135954999579),
 (17, 0.4472135954999579),
 (794, 0.4472135954999579),
 (1424, 0.408248290463863),
 (44, 0.40089186286863654),
 (109, 0.40089186286863654),
 (5, 0.3749999999999999)]

In [109]:
movie_name= []
for i in range(len(movie_list)):
    a = movie_list[i][0]
    movie_name.append(movie['movie_title'][a])

In [110]:
movie_name

['star trek into darkness',
 'captain america: the first avenger',
 'fantastic 4: rise of the silver surfer',
 'avengers: age of ultron',
 'the avengers',
 'the avengers',
 'tmnt',
 'terminator salvation',
 'terminator genisys',
 'john carter']

In [100]:
movie['movie_title'].nunique()

4916